In [ ]:
# @title Installing dependices
%pip install -U langgraph langchain_openai langchain_community
%pip install -qU langchain langchain-openai langchain-community langchain-experimental
!pip install -q langchain langchain-community langchain-google-genai
!pip install -q torch transformers accelerate bitsandbytes transformers sentence-transformers faiss-gpu pypdf python-dotenv
!pip install sqlalchemy-bigquery==1.11.0
!pip install google-cloud-bigquery-storage==2.26.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.6/50.6 kB 962.9 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 113.5/113.5 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.9/49.9 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 24.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 38.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 407.7/407.7 kB 24.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 296.7/296.7 kB 13.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 386.9/386.9 kB 15.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 23.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.0/78.0 kB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 325.2/325.2 kB 24.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/

In [ ]:
# @title Importing dependicies
import os
from langchain_google_genai import GoogleGenerativeAI
from langchain_community.utilities import SQLDatabase
from sqlalchemy import create_engine
from langchain_community.agent_toolkits import create_sql_agent
from langchain_openai import ChatOpenAI
from langchain_google_genai import ChatGoogleGenerativeAI
from pandas.io import gbq
from google.cloud import bigquery
import pandas as pd
import getpass
import dotenv
from typing import Any
from langchain_core.messages import ToolMessage
from langchain_core.runnables import RunnableLambda, RunnableWithFallbacks
from langchain_community.agent_toolkits import SQLDatabaseToolkit
from langchain_openai import ChatOpenAI
from langchain_core.tools import tool
from langchain_core.prompts import ChatPromptTemplate
from typing import Annotated, Literal

from langchain_core.messages import AIMessage
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain_openai import ChatOpenAI
from typing_extensions import TypedDict
from sqlalchemy import create_engine
from langchain_community.utilities import SQLDatabase
import warnings
from sqlalchemy import exc as sa_exc
import json

In [ ]:
from google.colab import auth
auth.authenticate_user()
print('Authenticated')

# Uploading and Handling Data Files

In [ ]:
import sqlite3
from google.colab import files
from langchain.sql_database import SQLDatabase

def upload_and_process_db():
    # upload file
    uploaded_file = files.upload()

    # get the file name
    filename = next(iter(uploaded_file))

    # file extension
    if filename.endswith(".csv"):
        # CSV files
        return process_csv_as_db(filename)
    elif filename.endswith(".db"):
        # SQLite DB files
        return connect_to_db(filename)
    else:
        raise ValueError("Unsupported file type. Please upload a CSV or SQLite DB file.")

def process_csv_as_db(csv_file):
    """
    Convert a CSV file into an SQLite in-memory database.
    Returns the URI to access the database.
    """
    # read the CSV file into a pandas DataFrame
    df = pd.read_csv(csv_file)

    # create a temporary SQLite file
    db_file = "temp_database.db"
    conn = sqlite3.connect(db_file)

    # convert the DataFrame into an SQLite table
    df.to_sql("uploaded_table", conn, index=False, if_exists='replace')

    print(f"CSV file '{csv_file}' uploaded and converted to SQLite file-based DB 'temp_database.db'.")

    # close the connection and return the file path (URI)
    conn.close()
    return f"sqlite:///{db_file}"

def connect_to_db(db_file):
    """
    Connect to an existing SQLite .db file and return the URI.
    """
    print(f"SQLite database file '{db_file}' connected.")

    return f"sqlite:///{db_file}"


In [ ]:
# upload and process the database file
db_uri = upload_and_process_db()

# create SQLDatabase object using the URI
db = SQLDatabase.from_uri(db_uri)

print("Database connected via URI:", db_uri)



In [ ]:
# create an instance of the ChatGoogleGenerativeAI model

llm = ChatGoogleGenerativeAI(
    model="gemini-1.5-flash",
    temperature=0.0,
    max_tokens=None,
    timeout=None,
    max_retries=2,

)

In [ ]:
# function to retrieve schema information from a database object or dictionary

def get_schema(db):
    # Check if the db object is an SQLDatabase instance with a method to get schema info
    if hasattr(db, 'get_table_info'):
        return db.get_table_info()
    elif isinstance(db, dict):
        # Check if schema information exists in the dictionary
        return db.get('schema', "Schema information not found in the db dictionary.")
    else:
        raise AttributeError("The provided db object does not contain schema or table information.")


In [ ]:
from langchain_core.prompts import ChatPromptTemplate

template = """Based on the tables schema below, write a GoogleSQL query that would answer the user's question:


Here is detailed steps about how to write efficient and well-structured SQL queries for Google BigQuery:
-Always reference your tables in the form project_id.dataset_id.table to avoid ambiguities in multi-project or multi-dataset queries:
-SELECT *
FROM `my_project.my_dataset.my_table`
WHERE ...;
-Avoid SELECT *: BigQuery charges based on the amount of data scanned. Explicitly specify the columns you need to avoid scanning unnecessary data.
-You should use SELECT DISTINCT when you want to return only unique values from a column or combination of columns, eliminating any duplicates.
- When user any question about customer try to retrieve much information about customer like his name,id,email...etc
-When user any question is yes or no question you provide the data which answer the question like :
Did the average order value increase in black friday 2023?
here you should divide problem to two main parts average order value before black friday and after black friday.
-You should write the Query only without any tags or ending with ''' write ready exucted query
-You Should use the DATE() function to convert the TIMESTAMP to DATE:
Few shot examples:

SQL Query:

"""

# create a ChatPromptTemplate object from the provided template

prompt = ChatPromptTemplate.from_template(template)

In [ ]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import ChatOpenAI

llm = llm

# start with a structured SQL prompt, use llm to generate the query text, and parse the result into a string

sql_chain = (


    prompt
    | llm.bind(stop=["SQL Query"])
    | StrOutputParser()
)


In [ ]:
# @title Query_check_prompt
query_check_system = """You are a google SQL expert with a strong attention to detail.
Double check the Google SQL query for common mistakes, including:
- Using NOT IN with NULL values
- Using UNION when UNION ALL should have been used
- Using BETWEEN for exclusive ranges
- Data type mismatch in predicates
- Properly quoting identifiers
- Using the correct number of arguments for functions
- Using the proper columns for joins
- Casting to the correct data type:
SELECT * FROM orders WHERE order_date = '2024-09-12';   Risk of incorrect format
SELECT * FROM orders WHERE order_date = DATE('2024-09-12'); coreccted format
SELECT
    o.referring_site AS referrer_source,
    SUM(o.total_price) AS total_sales
FROM
    shopify_fivetran_shopify.shopify__orders o
WHERE
    o.created_timestamp >= TIMESTAMP_SUB(CURRENT_TIMESTAMP(), INTERVAL 12 MONTH)
GROUP BY
    referrer_source
ORDER BY
    total_sales DESC
LIMIT 1    --TIMESTAMP_SUB does not support the MONTH date part when the argument is TIMESTAMP type
SELECT
    o.referring_site AS referrer_source,
    sum(o.total_price) AS total_sales
  FROM
    `smartycommerce.shopify_fivetran_shopify.shopify__orders` AS o
  WHERE o.created_timestamp >= TIMESTAMP_SUB(CURRENT_TIMESTAMP(), INTERVAL 365 DAY)
  GROUP BY 1
ORDER BY
  total_sales DESC
LIMIT 1; -- correct format

Use this style in defination of tables :
SELECT
    sum(shopify__orders.total_price)
  FROM
    `smartycommerce.shopify_fivetran_shopify.shopify__orders` AS shopify__orders
  WHERE shopify__orders.processed_timestamp BETWEEN '2024-06-24T00:00:00' AND '2024-06-24T23:59:59

- Remove ````googlesql`:** This is not a valid BigQuery syntax.
If there are any of the above mistakes, rewrite the query. If there are no mistakes, just reproduce the original query.

You will call the appropriate tool to execute the query after running this check."""

In [ ]:
# create a prompt template for checking and correcting Google SQL queries if needed

query_check_prompt = ChatPromptTemplate.from_template("{query}\n" + query_check_system)

query_check_chain = (
    query_check_prompt
    | llm
    | StrOutputParser()
)

In [ ]:
import re

def db_query_tool(query: str) -> str:
    """
    Executes a GoogleSQL query against the database and gets back the result.
    The function extracts the query starting from 'SELECT' till the end of the query body,
    ignoring any trailing characters like ''' after the actual query.
    Also removes the ```googlesql and ``` tags before execution.
    """
    # Remove the ```googlesql and ``` tags
    query = query.replace("```googlesql", "").replace("```", "").strip()

    # Use regex to capture everything from 'SELECT' until the end of the query
    clean_query_match = re.search(r"(SELECT.*?)(?:;|$)", query, re.DOTALL | re.IGNORECASE)

    if not clean_query_match:
        return "Error: No valid query found starting with 'SELECT'."

    clean_query = clean_query_match.group(1).strip()

    # Execute the clean query
    result = db.run_no_throw(clean_query)

    if not result:
        return "Error: Query failed. Please rewrite your query and try again."

    return result


In [ ]:
error_handling_template = """There was an error executing the following SQL query:
{query}

Error: {error_message}

Please rewrite the query to fix the error."""
error_handling_prompt = ChatPromptTemplate.from_template(error_handling_template)

error_handling_chain = (
    error_handling_prompt
    | llm
    | StrOutputParser()
)

In [ ]:
def full_query_chain(question: str, db_info: str):
    # Step 1: Generate SQL query
    generated_sql = sql_chain.invoke({"question": question, "db_info": db_info})

    # Step 2: Validate the generated SQL query
    validated_sql = query_check_chain.invoke({"query": generated_sql})

    # Step 3: Run the query and handle errors
    result = db_query_tool(validated_sql)

    if "Error" in result:
        # Step 4: If there's an error, pass it to the error-handling chain
        fixed_query = error_handling_chain.invoke({"query": validated_sql, "error_message": result})

        # Step 5: Try running the corrected query
        result = db_query_tool(fixed_query)

        # Return the fixed query and result, each formatted as a single line
        return '\n'.join([' '.join(fixed_query.split()), result])
    else:
        # Return the validated query and result, each formatted as a single line
        return '\n'.join([' '.join(validated_sql.split()), result])


In [ ]:
user_question = "Which time month had the most or least visitors"

In [ ]:
final_result = full_query_chain(user_question, db_info)

print(final_result)